# TinyMS ResNet50 服务器端教程

### 在本教程中，我们会演示获取ResNet50 ckpt文件和使用TinyMS API启动推理服务器的过程。

## 环境要求
 - Ubuntu: `18.04`
 - Python: `3.7.x`
 - Flask: `1.1.2`
 - MindSpore: `CPU-1.1.1`
 - TinyMS: `0.1.0`
 - numpy: `1.17.5`
 - opencv-python: `4.5.1.48`
 - Pillow: `8.1.0`
 - pip: `21.0.1`
 - requests: `2.18.4`
 
## 介绍

TinyMS是一个高级API，目的是让新手用户能够更加轻松地上手深度学习。TinyMS可以有效地减少用户在构建、训练、验证和推理一个模型过程中的操作次数。TinyMS也提供了教程和文档帮助开发者更好的上手和开发。

本教程中，由于使用CPU训练ResNet50模型过于耗时，所以本教程将直接提供训练好的ResNet50 ckpt文件。步骤包含3部分：获取ckpt文件、定义servable json和启动服务器，而进行推理操作的部分会在 `ResNet50_Client_tutorial_zh.ipynb`文件中进行讲解。


## 步骤

### 1. 获取ckpt文件

本教程提供两个由不同训练集训练而成的ResNet50模型，一个是由[ImageNet2012](http://www.image-net.org/challenges/LSVRC/2012/)数据集训练得来，另一个是由[cifar10](http://www.cs.toronto.edu/~kriz/cifar.html)数据集训练得来。启动服务器的前提条件是需要ResNet50 ckpt文件，可以点击[resnet_imagenet](https://ascend-tutorials.obs.cn-north-4.myhuaweicloud.com:443/resnet-50/ckpt_files/imagenet2012/resnet50.ckpt?AccessKeyId=HUXAJ6GDDQV6E9JLVZRG&Expires=1645083010&Signature=UmcUjCyDs/0jPRJCWu2e8xS5ooc%3D)下载ResNet50_imagenet ckpt文件，或者点击[resnet_cifar](https://ascend-tutorials.obs.cn-north-4.myhuaweicloud.com:443/resnet-50/ckpt_files/cifar10/resnet50.ckpt?AccessKeyId=HUXAJ6GDDQV6E9JLVZRG&Expires=1645083487&Signature=XUPGdTwctaw%2Bep04W3Bo4qxQYG0%3D)下载ResNet50_cifar ckpt文件


将ckpt文件保存到`/etc/tinyms/serving/resnet50_<dataset_name>/resnet50.ckpt`

### 2. 定义servable.json

定义servable json文件，servable json文件定义了servable名称，模型名称，模型格式和分类数量，以便后续推理使用。在该步骤中，将演示推理`ResNet50_imagenet2012`模型

In [ ]:
import os
import json
from tinyms.serving import start_server


servable_json = [{'name': 'resnet50_imagenet2012', 
                  'description': 'This servable hosts a resnet50 model predicting 10 classes of objects', 
                  'model': {
                      "name": "resnet50", 
                      "format": "ckpt", 
                      "class_num": 9}}]
os.chdir("/etc/tinyms/serving")
json_data = json.dumps(servable_json, indent=4)

with open('servable.json', 'w') as json_file:
    json_file.write(json_data)

如果用户想要运行`ResNet50_cifar10`模型，将上述代码中的`servable_json`字段替换成如下代码段：
```
servable_json = [{'name': 'resnet50_cifar10', 
                  'description': 'This servable hosts a resnet50 model predicting 10 classes of objects', 
                  'model': {
                      "name": "resnet50", 
                      "format": "ckpt", 
                      "class_num": 10}}]
```
并且请确保在修改servable文件前关停服务器，并重新运行。

### 3. 启动服务器

TinyMS推理是C/S（Client/Server）架构。TinyMS使用[Flask](https://flask.palletsprojects.com/en/1.1.x/)这个轻量化的网页服务器架构作为C/S通讯的基础架构。为了能够对模型进行推理，用户必须首先启动服务器。如果成功启动，服务器端会监听从地址127.0.0.1，端口号5000发送来的POST请求并且使用MindSpore作为后端来处理这些请求。后端会构建模型，运行推理并且返回结果给客户端

运行下列代码以启动服务器：

In [ ]:
# start(): start the server, call this function at the server side
start_server()

如果用户能够看到类似如下的输出:
```
* Serving Flask app "serving.server.server" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 ```
意味着服务器已经成功启动了，接下来的操作需要到`ResNet50_Client_tutorial_zh.ipynb`文件中完成

 
 ## 关闭服务器

重启服务器并使用另一种模型，点击左上方`Kernel`，再点击`Restart & Clear Output`，并替换`servable_json`代码段

关闭服务器，如果使用终端，可以直接CTRL + C关闭，如果使用Jupyter，点击上方`Kernel`再点击`Shutdown`